# Time Travel Problem

In [1]:
import os
import pandas as pd
import numpy as np
import math

In [2]:
ultrastocks_load = ['GE', 'ORCL', 'AMZN', 'NFLX', 'FB', 'TSLA', 'AAPL', 'AGN']

In [3]:
if not os.path.isdir("good_comps"):
    os.makedirs("good_comps")

# print these in a file
textfile = open("good_comps\\good_comps_large.txt", "w")
for element in ultrastocks_load:
    textfile.write(str(element) + "\n")
textfile.close()

In [4]:
# Function to see if the stock should be bought, intra-day traded or sold, see below.
def process_GE(df,stock,current_index,LOW_SEARCH,INTRA_SEARCH,HIGH_SEARCH,stocks,transactions,avail_money):
    
    mon_diff = 0.0
    
    # Calculate how many of this stock we currently have
    n = stocks[stock] if stock in stocks else 0

    # PART 1: FIRST CHECK IF WE CAN BUY AT LOW (and have some to spare)

    # find the lowest low of the next LOW_SEARCH entries (not dates, entries!)
    eff_df = df.loc[df['Local Index'] > current_index].loc[df['Local Index'] <= current_index+LOW_SEARCH]
    lowest_low = eff_df['Low'].min() # this is the lowest low
    ll_entry = eff_df.loc[eff_df['Low']==lowest_low]
    ll_idx = ll_entry['Local Index'].values[0] # this is the "local index" of the entry of the lowest low
    ll_dt = str(pd.to_datetime(ll_entry['Date'].values[0]).date()) # this is the date of the entry of the lowest low

    available_vol = math.floor(0.1*ll_entry['Volume'].values[0])

    vol_can_buy = math.floor(avail_money/lowest_low)

    if (n > 1 and vol_can_buy >= 2*n + 1) or (n == 0 and vol_can_buy > 0) or (n == 1 and vol_can_buy > 1):

        vol_to_buy = int(min(available_vol,n+1,vol_can_buy))

        transactions.append(f'{ll_dt} buy-low {stock} {vol_to_buy}')
        cost = vol_to_buy*lowest_low

        if stock in stocks:
            stocks[stock] += vol_to_buy
        else:
            stocks[stock] = vol_to_buy

        mon_diff = -cost

        return ll_idx, mon_diff

    # PART 2: IF WE CAN'T BUY, THEN WE PROCEED TO INTRA-DAY

    # find the best action of the next INTRA_SEARCH entries (not dates, entries!)
    eff_df = df.loc[df['Local Index'] > current_index].loc[df['Local Index'] <= current_index+INTRA_SEARCH]
    best_action = eff_df['Action'].max() # this is the best action
    best_entry = eff_df.loc[eff_df['Action']==best_action]
    best_idx = best_entry['Local Index'].values[0] # this is the "local index" of the entry of the best action
    best_dt = str(pd.to_datetime(best_entry['Date'].values[0]).date()) # this is the date of the entry of the best action

    # now we need to find if the action corresponds to BO-SH or BL-SC
    BOSH_counter = False # to make the check more easily further down
    if best_action == best_entry['BO-SH'].values[0]:
        price_pay = best_entry['Open'].values[0]
        price_earn = best_entry['High'].values[0]
        BOSH_counter = True
    else:
        price_pay = best_entry['Low'].values[0]
        price_earn = best_entry['Close'].values[0]

    # and finally we need to see if we can afford this intra-day
    available_vol = math.floor(0.1*best_entry['Volume'].values[0])

    vol_can_buy = math.floor(avail_money/price_pay)
    if vol_can_buy > 1:

        vol_to_buy = int(min(available_vol,n+1,vol_can_buy))

        if BOSH_counter: # this means we have a BO-SH

            transactions.append(f'{best_dt} buy-open {stock} {vol_to_buy}')
            cost = vol_to_buy*price_pay
            
            if f'{best_dt} buy-open {stock} {vol_to_buy}' == '1997-06-17 buy-open AAPL 1375':
                print(avail_money)
            
            transactions.append(f'{best_dt} sell-high {stock} {vol_to_buy}')
            earned = vol_to_buy*price_earn
            total_earnings = earned-cost

            mon_diff = total_earnings

        else: # this means we have a BL-SC

            transactions.append(f'{best_dt} buy-low {stock} {vol_to_buy}')
            cost = vol_to_buy*price_pay
            transactions.append(f'{best_dt} sell-close {stock} {vol_to_buy}')
            earned = vol_to_buy*price_earn
            total_earnings = earned-cost

            mon_diff = total_earnings

        return best_idx, mon_diff

    # PART 3 : IF WE CAN NEITHER BUY LOW, NOR INTRA-TRADE, THEN WE DON'T HAVE SUFFICIENT MONEY
    # WE THEREFORE NEED TO SELL SOME STOCK

    # find the best high of the next HIGH_SEARCH entries (not dates, entries!)
    eff_df = df.loc[df['Local Index'] > current_index].loc[df['Local Index'] <= current_index+HIGH_SEARCH]
    highest_high = eff_df['High'].max() # this is the highest high
    hh_entry = eff_df.loc[eff_df['High']==highest_high]
    hh_idx = hh_entry['Local Index'].values[0] # this is the "local index" of the entry of the highest high
    hh_dt = str(pd.to_datetime(hh_entry['Date'].values[0]).date()) # this is the date of the entry of the highest high

    available_vol = math.floor(0.1*hh_entry['Volume'].values[0])

    # Unless we only have one stock, in which case we need to sell it, we sell half of them
    # so we don't have to start buying again little by little
    vol_must_sell = n//2 if n != 1 else n

    if vol_must_sell > 0:

        vol_to_sell = int(min(available_vol,vol_must_sell))

        transactions.append(f'{hh_dt} sell-high {stock} {vol_to_sell}')
        earned = vol_to_sell*highest_high

        stocks[stock] -= vol_to_sell

        mon_diff = earned

        return hh_idx, mon_diff


    # if nothing has happened, we skip the max of LOW_SEARCH,INTRA_SEARCH,HIGH_SEARCH entries
    # in order to see if something changes, so we don't get stuck in an endless loop
    
    return current_index + max(LOW_SEARCH,INTRA_SEARCH,HIGH_SEARCH), mon_diff

In [5]:
# Function to get rid of a stock before the end of an era, see below.
def sell_GE(df,stock,current_index,end_index,stocks,transactions,avail_money):
    
    mon_diff = 0.0
    
    eff_df = df.loc[df['Local Index'] > current_index].loc[df['Local Index'] <= end_index]
    stock_to_sell = stocks[stock]
    
    # first check if we can sell all of our stock at once
    highest_high = eff_df['High'].max() # this is the highest high
    hh_entry = eff_df.loc[eff_df['High']==highest_high]
    available_vol = math.floor(0.1*hh_entry['Volume'].values[0])
    if stock_to_sell <= available_vol:
        
        hh_idx = hh_entry['Local Index'].values[0] # this is the "local index" of the entry of the highest high
        hh_dt = str(pd.to_datetime(hh_entry['Date'].values[0]).date()) # this is the date of the entry of the highest high
        
        transactions.append(f'{hh_dt} sell-high {stock} {stock_to_sell}')
        mon_diff = stock_to_sell*highest_high
        stocks[stock] -= stock_to_sell
        
        return hh_idx, mon_diff
        
    else:
        # In this case we need more than one days to sell the stock
        # so we will do it in a less optimized way
        entry = eff_df.loc[eff_df['Local Index']==current_index+1]
        available_vol = math.floor(0.1*entry['Volume'].values[0])
        
        price = entry['High'].values[0]
        entry_dt = str(pd.to_datetime(entry['Date'].values[0]).date())
        
        stock_sold = min(available_vol,stock_to_sell)
        
        transactions.append(f'{entry_dt} sell-high {stock} {stock_sold}')
        mon_diff = stock_sold*price
        stocks[stock] -= stock_sold
        
        return current_index+1, mon_diff

In [6]:
with open("good_comps\\good_comps_large.txt", "r") as file:
    comps = file.read().splitlines()

direc = "Stocks\\"
good_stocks = []
stocks_dict = {}
g_counter = 0

for filename in os.listdir(direc):
    statinfo = os.stat(direc+filename)
    
    if filename.split('.us')[0].upper() in comps:
        data = pd.read_csv(direc + filename, header=0, parse_dates=["Date"])
        data.insert(1, "Company Index", g_counter)
        data['Stock name'] = filename.split('.us')[0].upper()
        
        data.drop(data[data['Volume'] < 10].index, inplace=True)
        
        if filename.split('.us')[0].upper() == 'GE':
            gedf = data

            # This column is useless for our analysis, explain more in the text
            gedf.drop(['OpenInt'], axis=1, inplace=True)

            gedf.reset_index(inplace=True)
            gedf.rename(columns={'index': 'Local Index'}, inplace=True)
            
            # New columns for intra day buying
            gedf['BO-SH'] = gedf['High']-gedf['Open']
            gedf['BL-SC'] = gedf['Close']-gedf['Low']
            gedf['Action'] = gedf[['BO-SH','BL-SC']].max(axis=1) # In order to choose in each case what to do.
            
            g_counter += 1
        else:
            good_stocks.append(data)
            stocks_dict[g_counter] = filename.split('.us')[0].upper()
            g_counter += 1
            
print("Added a total of {} stocks.".format(g_counter))

df = pd.concat(good_stocks)

# This column is useless for our analysis, explain more in the text
df.drop(['OpenInt'], axis=1, inplace=True)

df.reset_index(inplace=True)
df.rename(columns={'index': 'Local Index'}, inplace=True)

# New columns for intra day buying
df['BO-SH'] = df['High']-df['Open']
df['BL-SC'] = df['Close']-df['Low']
df['Action'] = df[['BO-SH','BL-SC']].max(axis=1) # In order to choose in each case what to do.

Added a total of 8 stocks.


In [7]:
lowest_dates = {}
for comp in stocks_dict.values():
    Company = df.loc[df['Stock name'] == comp]
    st_dt = Company[Company['Local Index']==Company['Local Index'].min()]['Date'].values[0]
    
    lowest_dates[comp] = pd.to_datetime(st_dt)
    
lowest_dates = {k: v for k, v in sorted(lowest_dates.items(), key=lambda item: item[1])} # sort by date
when_start_next = lowest_dates[list(lowest_dates)[0]]

print(f"The first stock after GE is {list(lowest_dates)[0]} and starts trading in {str(pd.to_datetime(when_start_next).date())}.")

The first stock after GE is AAPL and starts trading in 1984-09-07.


In [8]:
# Parameters
# --------------------------------------------------
LOW_SEARCH = 10
INTRA_SEARCH = 1
HIGH_SEARCH = 10

transactions = []
stocks = {}
ge_money = 1.0 # We begin with one dollar
# --------------------------------------------------

# This starts summing over eras
# each era handles one type of stock
stock = 'GE'
        
curr_idx = gedf['Local Index'].min()
# The ending index corresponds to the date that 
end_idx = gedf[gedf['Date']<=when_start_next]['Local Index'].max()-1
    
# Working with the stock along the df that corresponds to it
while (curr_idx <= end_idx):
        
    searches_left = end_idx-curr_idx
    # Check if it's coming close to the end of the era, so we need to sell stock
    if searches_left < min(LOW_SEARCH,INTRA_SEARCH,HIGH_SEARCH)+10 and stock in stocks:
        # sell all of these stocks to buy the new ones
        if stocks[stock] > 0:
            new_idx, money = sell_GE(gedf,stock,curr_idx,end_idx,stocks,transactions,ge_money)
            ge_money += money
            curr_idx = new_idx
        else:
            break
    else:
        new_idx, money = process_GE(gedf,stock,curr_idx,LOW_SEARCH,INTRA_SEARCH,HIGH_SEARCH,stocks,transactions,ge_money)
        ge_money += money
        curr_idx = new_idx
        
ge_trans = len(transactions)

print(f"GE finished trading after {ge_trans} transactions. You currently have {ge_money:.2f}$ in your wallet.")
print("Proceeding with the other stocks.")

GE finished trading after 10278 transactions. You currently have 46576042.09$ in your wallet.
Proceeding with the other stocks.


In [9]:
# The Initial conditions are now altered
avail_money = math.floor(ge_money) # After GE

start_date = str(pd.to_datetime(when_start_next).date()) # The date stocks begin trading after GE
end_date = str(df['Date'].max().date())
dates = pd.date_range(start_date, end_date)

ultrastocks = {'AMZN' : 0, 'FB' : 0, 'ORCL' : 0, 'NFLX' : 0, 'TSLA' : 0, 'AAPL' : 0, 'AGN' : 0}
ultrathresholds = {'AMZN' : 50, 'FB' : 140, 'ORCL' : 60, 'NFLX' : 140, 'TSLA' : 80, 'AAPL' : 49, 'AGN' : 70}
ultradates = {}

for comp in ultrastocks:
    Company = df.loc[df['Stock name'] == comp]
    comp_lowest = Company['Low'].min()
    st_dt = Company[Company['Low']==comp_lowest].Date.values[0]
    st_date = pd.date_range(st_dt, st_dt)
    ultradates[comp] = st_date[0]
    print(f'The optimal starting date to buy stocks from {comp} is {str(st_date[0].date())}.')
    
# Testing proved that bypassing this info works better,
# with the exception of FB
for key in ultradates:
    if key == 'FB':
        continue
    else:
        ultradates[key] = dates[0]

The optimal starting date to buy stocks from AMZN is 1997-05-22.
The optimal starting date to buy stocks from FB is 2012-09-04.
The optimal starting date to buy stocks from ORCL is 1990-10-31.
The optimal starting date to buy stocks from NFLX is 2005-03-24.
The optimal starting date to buy stocks from TSLA is 2010-11-19.
The optimal starting date to buy stocks from AAPL is 1985-08-15.
The optimal starting date to buy stocks from AGN is 1993-02-22.


In [10]:
# Algorithm for intra-day trading - edited
for x in dates:
    
    # These are added to buy more AAPL stock at moments when
    # there is lots of capital and the stock is still very cheap
    if x == pd.to_datetime('1994-01-01'):
        ultrathresholds['AAPL'] = 61
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('1996-01-01'):
        ultrathresholds['AAPL'] = 74
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('1997-01-01'):
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('1998-01-01'):
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('1999-01-01'):
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('2000-01-01'):
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('2001-01-01'):
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('2002-05-01'):
        ultrastocks['AAPL'] = 0
    if x == pd.to_datetime('2003-05-01'):
        ultrastocks['AAPL'] = 0
    
    datex = str(x.date())
    earned_money = 0.0 # for this day
    spent_money = 0.0 # for this day
    eff_df = df.loc[df['Date']==x].sort_values(by=['Action'], ascending=False)
    if not eff_df.empty: # Maybe there are no stocks at all in this day
        for i in range(eff_df.shape[0]): # summation on stocks available this day
            stock = eff_df.iloc[i,8]
            avail_vol = math.floor(0.1*eff_df.iloc[i,7])
            
            cost = 0.0
            earned = 0.0
            
            # In the case of ultrastocks we only buy in the beginning
            if stock in ultrastocks and ultrastocks[stock] < ultrathresholds[stock] and x >= ultradates[stock]:
                price_pay = eff_df.iloc[i,5] # Buy low

                n = stocks[stock] if stock in stocks else 0

                vol_can_buy = int(min(math.floor(avail_money/price_pay),n+1))

                if vol_can_buy >= avail_vol:
                    transactions.append('{} buy-low {} {}'.format(datex,stock,avail_vol))
                    ultrastocks[stock] += 1
                    cost = avail_vol*price_pay
                    if stock in stocks:
                        stocks[stock] += avail_vol
                    else:
                        stocks[stock] = avail_vol
                elif vol_can_buy > 0:
                    transactions.append('{} buy-low {} {}'.format(datex,stock,vol_can_buy))
                    ultrastocks[stock] += 1
                    cost = vol_can_buy*price_pay
                    if stock in stocks:
                        stocks[stock] += vol_can_buy
                    else:
                        stocks[stock] = vol_can_buy
            else:
                if eff_df.iloc[i,11] == eff_df.iloc[i,10]: # BL-SC

                    # BUY LOW PROCESS
                    price_pay = eff_df.iloc[i,5]

                    n = stocks[stock] if stock in stocks else 0

                    vol_can_buy = int(min(math.floor(avail_money/price_pay),n+1))

                    if vol_can_buy >= avail_vol:
                        transactions.append('{} buy-low {} {}'.format(datex,stock,avail_vol))
                        cost = avail_vol*price_pay
                        if stock in stocks:
                            stocks[stock] += avail_vol
                        else:
                            stocks[stock] = avail_vol
                    elif vol_can_buy > 0:
                        transactions.append('{} buy-low {} {}'.format(datex,stock,vol_can_buy))
                        cost = vol_can_buy*price_pay
                        if stock in stocks:
                            stocks[stock] += vol_can_buy
                        else:
                            stocks[stock] = vol_can_buy

                    # SELL CLOSE PROCESS
                    price_earn = eff_df.iloc[i,6] # Price for Close

                    if stock in stocks:
                        #vol_have = stocks[stock]
                        vol_have = vol_can_buy if vol_can_buy < avail_vol else avail_vol

                        if vol_have >= avail_vol:
                            transactions.append('{} sell-close {} {}'.format(datex,stock,avail_vol))
                            earned = avail_vol*price_earn
                            stocks[stock] -= avail_vol
                        elif vol_have > 0:
                            transactions.append('{} sell-close {} {}'.format(datex,stock,vol_have))
                            earned = vol_have*price_earn
                            stocks[stock] -= vol_have

                else: # BO-SH

                    # BUY OPEN PROCESS
                    price_pay = eff_df.iloc[i,3] # Price for Open

                    n = stocks[stock] if stock in stocks else 0

                    vol_can_buy = min(math.floor(avail_money/price_pay),n+1)

                    if vol_can_buy >= avail_vol:
                        transactions.append('{} buy-open {} {}'.format(datex,stock,avail_vol))
                        cost = avail_vol*price_pay
                        if stock in stocks:
                            stocks[stock] += avail_vol
                        else:
                            stocks[stock] = avail_vol
                    elif vol_can_buy > 0:
                        transactions.append('{} buy-open {} {}'.format(datex,stock,vol_can_buy))
                        cost = vol_can_buy*price_pay
                        if stock in stocks:
                            stocks[stock] += vol_can_buy
                        else:
                            stocks[stock] = vol_can_buy

                    # SELL HIGH PROCESS
                    price_earn = eff_df.iloc[i,4] # Price for High

                    if stock in stocks:
                        #vol_have = stocks[stock]
                        vol_have = vol_can_buy if vol_can_buy < avail_vol else avail_vol

                        if vol_have >= avail_vol:
                            transactions.append('{} sell-high {} {}'.format(datex,stock,avail_vol))
                            earned = avail_vol*price_earn
                            stocks[stock] -= avail_vol
                        elif vol_have > 0:
                            transactions.append('{} sell-high {} {}'.format(datex,stock,vol_have))
                            earned = vol_have*price_earn
                            stocks[stock] -= vol_have

            spent_money += cost
            earned_money += earned
            avail_money -= cost
     
    # At the end of the day:
    avail_money += earned_money

In [11]:
# Some diagnostics
print(f"An extra {len(transactions)-ge_trans} transactions were performed, totalling to {len(transactions)} including the GE transactions.")
print(f"The money currently available in your wallet is {avail_money:.2f}$.")
for key in stocks:
    if stocks[key] > 0:
          print(f"There are still {stocks[key]} stocks left for {key}.")

An extra 65334 transactions were performed, totalling to 75612 including the GE transactions.
The money currently available in your wallet is 35560267808.00$.
There are still 5452183334 stocks left for AAPL.
There are still 25757802 stocks left for ORCL.
There are still 421908 stocks left for AGN.
There are still 12537572 stocks left for AMZN.
There are still 137022104 stocks left for NFLX.
There are still 4232463 stocks left for TSLA.
There are still 707565576 stocks left for FB.


In [12]:
# now we need to find a way to incorporate the final stocks
# during the final era only intra-day tradings happen, so we
# can just replace one such trading with sales.

transactions_ed = transactions.copy()
indices_to_ignore = []
extra_money = 0.0

for stock in stocks:
    
    # how much of this stock we have left
    leftovers = stocks[stock]
    
    # We search for the corresponding stock df
    stock_df = df.loc[df['Stock name'] == stock]

    edits = [] # what to replace inside the transactions list
    cntr = -1 # counter
    while leftovers > 0:

        volume = math.floor(0.1*stock_df.iloc[cntr][7])
        price = stock_df.iloc[cntr][4] # high price
        datex = str(stock_df.iloc[cntr][1].date())
        if leftovers-volume < 0:
            edits.append(f'{datex} sell-high {stock} {leftovers}')
            extra_money += leftovers*price
            break
        else:
            edits.append(f'{datex} sell-high {stock} {volume}')
            extra_money += volume*price
        leftovers -= volume
        cntr -= 1

    goback = len(edits) # how many edits have to be performed

    cntr = -1
    indices_to_edit = []

    while goback > 0:

        if f'sell-high {stock}' in transactions[cntr] or f'sell-close {stock}' in transactions[cntr]:
            indices_to_edit.append(transactions.index(transactions[cntr]))
            indices_to_ignore.append(transactions.index(transactions[cntr-1]))
            cntr -= 2
            goback -= 1
        else:
            cntr -=1

    i = 0
    for idx in indices_to_edit:
        transactions_ed[idx] = edits[i]
        i += 1

In [13]:
print(f"The extra money received from selling the remaining stocks was {extra_money:.2f}$.")
print(f"Approximately, the total money at the end of all trading is {extra_money + avail_money}$.")
print(f"For an exact calculation refer to the valuation script.")

The extra money received from selling the remaining stocks was 680138564972.73$.
Approximately, the total money at the end of all trading is 715698832780.7262$.
For an exact calculation refer to the valuation script.


In [14]:
# print on the file
ignored_indices = len(indices_to_ignore)
textfile = open("large.txt", "w")
textfile.write(str(len(transactions_ed)-ignored_indices) + "\n")
for i, element in enumerate(transactions_ed):
    if i not in set(indices_to_ignore):
        textfile.write(element + "\n")
textfile.close()